In [1]:
from aicsimageio import AICSImage
#import czifile
import matplotlib.pyplot as plt
import numpy as np
import math
from PIL import Image
import cv2
from pylibCZIrw import czi
from tqdm import tqdm
import pickle
import random
import keras
#from stardist.models import StarDist2D
from skimage.filters import threshold_multiotsu
import skimage
import operator
import scipy
import xgboost as xgb
import sklearn
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Dropout
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
import pandas as pd
from sklearn.feature_selection import f_classif
import copy
from skimage.morphology import disk
#import pywt

C:\Users\c23115040\AppData\Local\anaconda3\envs\env\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.2 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "
C:\Users\c23115040\AppData\Local\anaconda3\envs\env\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.2 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "
C:\Users\c23115040\AppData\Local\anaconda3\envs\env\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.2 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "
C:\Users\c23115040\AppData\Local\anaconda3\envs\env\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.2 when it was built against 1.14.5, this

In [2]:
lst = open("C:/Users/c23115040/OneDrive - Cardiff University/Desktop/TMA Data/list.txt", "r")
lst = lst.read()
lst = lst.split("\n")
list2 = []
for i in range(len(lst)):
    if lst[i][12:15] == '0.6':
        list2.append(lst[i])
lst = list2

In [ ]:
for i in range(len(lst)):
    img = AICSImage("C:/Users/c23115040/OneDrive - Cardiff University/Desktop/TMA Data/" + lst[i] + ".czi")
    DRAQ5 = img.data[0,0,0,:,:]
    DRAQ5_mean = np.mean(DRAQ5)
    DRAQ5_gamma = math.log(2048)/math.log(DRAQ5_mean)
    DRAQ5_comp = (DRAQ5- np.min(DRAQ5))/(np.max(DRAQ5) - np.min(DRAQ5))
    #...calibration
    DRAQ5_comp = np.power(DRAQ5_comp, DRAQ5_gamma)
    Eosin = img.data[0,1,0,:,:]
    Eosin_mean = np.mean(Eosin)
    Eosin_gamma = math.log(2048)/math.log(Eosin_mean)
    Eosin_comp = (Eosin- np.min(Eosin))/(np.max(Eosin) - np.min(Eosin))
    #...calibration
    Eosin_comp = np.power(Eosin_comp, Eosin_gamma)
    DRAQ5_comp_mean = np.mean(DRAQ5_comp)
    Eosin_comp_mean = np.mean(Eosin_comp)
    average = (DRAQ5_comp_mean + Eosin_comp_mean)/2
    DRAQ5_scale = average/DRAQ5_comp_mean
    Eosin_scale = average/Eosin_comp_mean
    DRAQ5_comp = DRAQ5_scale*DRAQ5_comp
    Eosin_comp = Eosin_scale*Eosin_comp
    row, column = DRAQ5.shape
    R = np.ones((row, column)) - (1-0.24)*DRAQ5_comp - (1-0.88)*Eosin_comp
    R[R<0] = 0
    R[R>1] = 1
    R = (65535*((R- np.min(R))/(np.max(R) - np.min(R)))).astype(np.uint16)
    G = np.ones((row, column)) - (1-0.21)*DRAQ5_comp - (1-0.27)*Eosin_comp
    G[G<0] = 0
    G[G>1] = 1
    G = (65535*((G- np.min(G))/(np.max(G) - np.min(G)))).astype(np.uint16)
    B = np.ones((row, column)) - (1-0.62)*DRAQ5_comp - (1-0.66)*Eosin_comp
    B[B<0] = 0
    B[B>1] = 1
    B = (65535*((B- np.min(B))/(np.max(B) - np.min(B)))).astype(np.uint16)
    RGB = np.stack((R,G,B), axis = 2)
    with czi.create_czi("C:/Users/c23115040/Downloads/H&E_Simulated/" + lst[i] + "H&E.czi", exist_ok=True) as czidoc_w:
        for c in tqdm(range(3)):
            czidoc_w.write(
                data=RGB[:,:,c],
            plane={"C": c}
        )
        czidoc_w.write_metadata(
            document_name=lst[i][:-4] + "H&E.tiff",
            channel_names={0: "Red", 1: "Green", 2: "Blue"},
            scale_x=img.physical_pixel_sizes.X*0.000001,
            scale_y=img.physical_pixel_sizes.Y*0.000001,
            scale_z=img.physical_pixel_sizes.Z*0.000001,
        )

In [6]:
lst = open("C:/Users/c23115040/OneDrive - Cardiff University/Desktop/AMW/June 21/list.txt", "r")
lst = lst.read()
lst = lst.split("\n")

In [ ]:
for i in range(len(lst)):
    img = AICSImage("C:/Users/c23115040/OneDrive - Cardiff University/Desktop/AMW/June 21/CA4_I83_TMA_" + lst[i] + ".czi")
    DRAQ5 = img.data[0,1,0,:,:]
    DRAQ5_mean = np.mean(DRAQ5)
    DRAQ5_gamma = math.log(2048)/math.log(DRAQ5_mean)
    DRAQ5_comp = (DRAQ5- np.min(DRAQ5))/(np.max(DRAQ5) - np.min(DRAQ5))
    #...calibration
    DRAQ5_comp = np.power(DRAQ5_comp, DRAQ5_gamma)
    Eosin = img.data[0,0,0,:,:]
    Eosin_mean = np.mean(Eosin)
    Eosin_gamma = math.log(2048)/math.log(Eosin_mean)
    Eosin_comp = (Eosin- np.min(Eosin))/(np.max(Eosin) - np.min(Eosin))
    #...calibration
    Eosin_comp = np.power(Eosin_comp, Eosin_gamma)
    DRAQ5_comp_mean = np.mean(DRAQ5_comp)
    Eosin_comp_mean = np.mean(Eosin_comp)
    average = (DRAQ5_comp_mean + Eosin_comp_mean)/2
    DRAQ5_scale = average/DRAQ5_comp_mean
    Eosin_scale = average/Eosin_comp_mean
    DRAQ5_comp = DRAQ5_scale*DRAQ5_comp
    Eosin_comp = Eosin_scale*Eosin_comp
    row, column = DRAQ5.shape
    R = np.ones((row, column)) - (1-0.24)*DRAQ5_comp - (1-0.88)*Eosin_comp
    R[R<0] = 0
    R[R>1] = 1
    R = (65535*((R- np.min(R))/(np.max(R) - np.min(R)))).astype(np.uint16)
    G = np.ones((row, column)) - (1-0.21)*DRAQ5_comp - (1-0.27)*Eosin_comp
    G[G<0] = 0
    G[G>1] = 1
    G = (65535*((G- np.min(G))/(np.max(G) - np.min(G)))).astype(np.uint16)
    B = np.ones((row, column)) - (1-0.62)*DRAQ5_comp - (1-0.66)*Eosin_comp
    B[B<0] = 0
    B[B>1] = 1
    B = (65535*((B- np.min(B))/(np.max(B) - np.min(B)))).astype(np.uint16)
    RGB = np.stack((R,G,B), axis = 2)
    with czi.create_czi("C:/Users/c23115040/OneDrive - Cardiff University/Desktop/H&E_Simulated2/CA4_I83_TMA_" + lst[i] + "H&E.czi", exist_ok=True) as czidoc_w:
        for c in tqdm(range(3)):
            czidoc_w.write(
                data=RGB[:,:,c],
            plane={"C": c}
        )
        czidoc_w.write_metadata(
            document_name=lst[i][:-4] + "H&E.tiff",
            channel_names={0: "Red", 1: "Green", 2: "Blue"},
            scale_x=img.physical_pixel_sizes.X*0.000001,
            scale_y=img.physical_pixel_sizes.Y*0.000001,
            scale_z=img.physical_pixel_sizes.Z*0.000001,
        )

In [2]:
lst = open("C:/Users/c23115040/OneDrive - Cardiff University/Desktop/AMW/July 3/list.txt", "r")
lst = lst.read()
lst = lst.split("\n")

In [ ]:
for i in range(len(lst)):
    img = AICSImage("C:/Users/c23115040/OneDrive - Cardiff University/Desktop/AMW/July 3/" + lst[i][:-1] + ".czi")
    DRAQ5 = img.data[0,1,0,:,:]
    DRAQ5_mean = np.mean(DRAQ5)
    DRAQ5_gamma = math.log(2048)/math.log(DRAQ5_mean)
    DRAQ5_comp = (DRAQ5- np.min(DRAQ5))/(np.max(DRAQ5) - np.min(DRAQ5))
    #...calibration
    DRAQ5_comp = np.power(DRAQ5_comp, DRAQ5_gamma)
    Eosin = img.data[0,0,0,:,:]
    Eosin_mean = np.mean(Eosin)
    Eosin_gamma = math.log(2048)/math.log(Eosin_mean)
    Eosin_comp = (Eosin- np.min(Eosin))/(np.max(Eosin) - np.min(Eosin))
    #...calibration
    Eosin_comp = np.power(Eosin_comp, Eosin_gamma)
    DRAQ5_comp_mean = np.mean(DRAQ5_comp)
    Eosin_comp_mean = np.mean(Eosin_comp)
    average = (DRAQ5_comp_mean + Eosin_comp_mean)/2
    DRAQ5_scale = average/DRAQ5_comp_mean
    Eosin_scale = average/Eosin_comp_mean
    DRAQ5_comp = DRAQ5_scale*DRAQ5_comp
    Eosin_comp = Eosin_scale*Eosin_comp
    row, column = DRAQ5.shape
    R = np.ones((row, column)) - (1-0.24)*DRAQ5_comp - (1-0.88)*Eosin_comp
    R[R<0] = 0
    R[R>1] = 1
    R = (65535*((R- np.min(R))/(np.max(R) - np.min(R)))).astype(np.uint16)
    G = np.ones((row, column)) - (1-0.21)*DRAQ5_comp - (1-0.27)*Eosin_comp
    G[G<0] = 0
    G[G>1] = 1
    G = (65535*((G- np.min(G))/(np.max(G) - np.min(G)))).astype(np.uint16)
    B = np.ones((row, column)) - (1-0.62)*DRAQ5_comp - (1-0.66)*Eosin_comp
    B[B<0] = 0
    B[B>1] = 1
    B = (65535*((B- np.min(B))/(np.max(B) - np.min(B)))).astype(np.uint16)
    RGB = np.stack((R,G,B), axis = 2)
    with czi.create_czi("C:/Users/c23115040/OneDrive - Cardiff University/Desktop/Pathologist Intercorrelation Analysis/Synthetic_H&E/" + lst[i] + "H&E.czi", exist_ok=True) as czidoc_w:
        for c in tqdm(range(3)):
            czidoc_w.write(
                data=RGB[:,:,c],
            plane={"C": c}
        )
        czidoc_w.write_metadata(
            document_name=lst[i][:-4] + "H&E.tiff",
            channel_names={0: "Red", 1: "Green", 2: "Blue"},
            scale_x=img.physical_pixel_sizes.X*0.000001,
            scale_y=img.physical_pixel_sizes.Y*0.000001,
            scale_z=img.physical_pixel_sizes.Z*0.000001,
        )